In [1]:
import cv2
import numpy as np
import mediapipe as mp


# Create an instance of the Hands model
hand_instance = mp.solutions.hands


# Initialize Mediapipe Hands model
hands = hand_instance.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.5)


# Load the cascade classifier
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Open the video file
cap = cv2.VideoCapture('final.mp4')

# Get the frame rate of the video
frame_rate = cap.get(cv2.CAP_PROP_FPS)
print("Frame rate:", frame_rate)

# Set the frame rate of the video
cap.set(cv2.CAP_PROP_FPS, 30)

# Get the total number of frames in the video
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(num_frames)

# Initialize variables for tracking the head position and shoulder positions
head_pos = None
lt_shoulder_frame = None
rt_shoulder_frame = None


# Loop through each frame in the video
# Process each frame in the video
for frame_num in range(num_frames):
    
    # Read the next frame
    boolean, frame = cap.read()

    # If there are no more frames, exit the loop
    if not boolean:
        break
    # Convert frame to RGB for Mediapipe
    rgb_frame_conv = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detect hands in the frame
    had_detection = hands.process(rgb_frame_conv)
    
    # Draw bounding boxes around hands
    if had_detection.multi_hand_landmarks:
        for hand_landmarks in had_detection.multi_hand_landmarks:
            # Get landmarks for the hand
            landmarks = hand_landmarks.landmark
            
            a = min([landmark.x for landmark in landmarks])
            b = min([landmark.y for landmark in landmarks])
            c = max([landmark.x for landmark in landmarks])
            d = max([landmark.y for landmark in landmarks])
            # Get bounding box coordinates for the hand
            left_coord = int(a * frame.shape[1])
            top_coord = int(b * frame.shape[0])
            right_coord = int(c * frame.shape[1])
            bottom_coord = int(d * frame.shape[0])
            
            # Draw bounding box
            cv2.rectangle(frame, (left_coord, top_coord), (right_coord, bottom_coord), (0, 255, 0), 2)

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # If a face is detected, track its position
    if len(faces) > 0:
        # Use the first detected face
        (x, y, w, h) = faces[0]

        # Update the head position
        head_pos = (x, y, w, h)
        l = round(w/4)

        # Calculate the position of the shoulders based on the head position
        # we can adjust these values to fit the size and shape of your video frames
        lt_shoulder_frame = (x - w // 2, y + h // 2)
        rt_shoulder_frame = (x + w, y + h // 2)

        # Extract the left and right shoulder regions only for frames
    if lt_shoulder_frame is not None and rt_shoulder_frame is not None:
        (x1, y1) = lt_shoulder_frame
        (x2, y2) = rt_shoulder_frame
        left_shoulder = frame[y1:y1+h, x1:x1+w//2]
        right_shoulder = frame[y2:y2+h, x2:x2+w//2]

        # Convert the shoulder regions to grayscale
        left_shoulder_gray = cv2.cvtColor(left_shoulder, cv2.COLOR_BGR2GRAY)
        right_shoulder_gray = cv2.cvtColor(right_shoulder, cv2.COLOR_BGR2GRAY)

        # Reduce noise in the grayscale images
        left_shoulder_blur = cv2.GaussianBlur(left_shoulder_gray, (5, 5), 0)
        right_shoulder_blur = cv2.GaussianBlur(right_shoulder_gray, (5, 5), 0)

        # Detect edges in the grayscale images using Canny edge detection
        left_edges = cv2.Canny(left_shoulder_blur, 50, 150)
        right_edges = cv2.Canny(right_shoulder_blur, 50, 150)

        # Detect lines in the edge images using Hough lines
        # here based on the video and edge detections we need to adjust the threshhold, minimum and maximum length of the line
        left_lines = cv2.HoughLinesP(left_edges, rho=1, theta=np.pi/180, threshold=35, minLineLength=30, maxLineGap=20)
        right_lines = cv2.HoughLinesP(right_edges, rho=1, theta=np.pi/180, threshold=35, minLineLength=30, maxLineGap=20)

        # Combine the left and right lines into a single list of lines
        lines = []
        if left_lines is not None:
            lines.extend(left_lines)
        if right_lines is not None:
            lines.extend(right_lines)
        print(lines)

        # Define the range of slopes for the shoulder lines
        shoulder_slope_min = -5.0
        shoulder_slope_max = 0.0
        previous_slope_left = 0
        previous_slope_right = 0
        shrug_detected = False

        # Draw the lines as contours on the shoulder regions(right as left - mirroring)
        for line in lines:
            x1, y1, x2, y2 = line[0]

            slope = (y2 - y1) / (x2 - x1)
            print(slope)
            print(slope-previous_slope_left)
            print("==========================")
            if abs(slope - previous_slope_left) > 0.9:
                shrug_detected = True

            # Update previous slope
            previous_slope_left = slope
            if shoulder_slope_min <= slope <= shoulder_slope_max:
                m1 = (y2-y1)/(x2-x1)
                cv2.line(left_shoulder, (x1, y1), (x2, y2), (0, 0, 255), 2)

        shoulder_slope_min = 0.0
        shoulder_slope_max = 5
        for line in lines:
            x1, y1, x2, y2 = line[0]
            slope = (y2 - y1) / (x2 - x1)

            # Check if slope has changed significantly
            print(slope-previous_slope_right)
            print("==========================")
            if abs(slope - previous_slope_right) > 0.9:
                shrug_detected = True

            # Update previous slope
            previous_slope_right = slope
            if shoulder_slope_min <= slope <= shoulder_slope_max:
                cv2.line(right_shoulder, (x1, y1), (x2, y2), (0, 0, 255), 2)
         #Add text indicating whether a shrug was detected in the current frame
        if shrug_detected:
            cv2.putText(frame, f"Shrug detected in frame {frame_num}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (252, 3, 215), 2)

        # Reset shrug_detected flag for next frame
        shrug_detected = False
        
        # Draw rectangles around the head and shoulders and indicate shoulder shrugging
        if head_pos is not None:
            # Draw a rectangle around the head
            (x, y, w, h) = head_pos
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            # Draw rectangles around the left and right shoulders
    #         if lt_shoulder_frame is not None:
    #             (x, y) = lt_shoulder_frame
    #             cv2.rectangle(frame, (x, y), (x + w // 2, y + h), (255, 0, 0), 2)
    #         if rt_shoulder_frame is not None:
    #             (x, y) = rt_shoulder_frame
    #             cv2.rectangle(frame, (x, y), (x + w//2, y + h), (0, 0, 255), 2)

        # Display the frame
        cv2.imshow("Video", frame)
        cv2.waitKey(1)
    
# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()

Frame rate: 30.0
1817
[array([[ 3, 97, 48, 84]], dtype=int32), array([[ 4, 82, 50, 90]], dtype=int32)]
-0.28888888888888886
-0.28888888888888886
0.17391304347826086
0.4628019323671497
-0.28888888888888886
0.4628019323671497
[array([[ 5, 98, 57, 83]], dtype=int32)]
-0.28846153846153844
-0.28846153846153844
-0.28846153846153844
[array([[ 0, 99, 41, 86]], dtype=int32), array([[10, 84, 53, 91]], dtype=int32)]
-0.3170731707317073
-0.3170731707317073
0.16279069767441862
0.47986386840612594
-0.3170731707317073
0.47986386840612594
[array([[10, 96, 52, 84]], dtype=int32), array([[ 5, 82, 54, 91]], dtype=int32)]
-0.2857142857142857
-0.2857142857142857
0.1836734693877551
0.4693877551020408
-0.2857142857142857
0.4693877551020408
[]
[array([[ 8, 97, 43, 86]], dtype=int32), array([[ 7, 82, 58, 92]], dtype=int32)]
-0.3142857142857143
-0.3142857142857143
0.19607843137254902
0.5103641456582633
-0.3142857142857143
0.5103641456582633
[]
[array([[ 5, 81, 56, 91]], dtype=int32)]
0.19607843137254902
0.19607

[]
[array([[17, 73, 63, 69]], dtype=int32)]
-0.08695652173913043
-0.08695652173913043
-0.08695652173913043
[]
[array([[ 30,  87,  40, 130]], dtype=int32), array([[25, 60, 64, 60]], dtype=int32)]
4.3
4.3
0.0
-4.3
4.3
-4.3
[array([[ 28,  85,  37, 125]], dtype=int32)]
4.444444444444445
4.444444444444445
4.444444444444445
[array([[11, 89, 12, 55]], dtype=int32)]
-34.0
-34.0
-34.0
[array([[ 10, 119,  15,  51]], dtype=int32), array([[ 31,  71,  37, 124]], dtype=int32)]
-13.6
-13.6
8.833333333333334
22.433333333333334
-13.6
22.433333333333334
[array([[ 33,  67,  37, 125]], dtype=int32), array([[12, 88, 15, 48]], dtype=int32)]
14.5
14.5
-13.333333333333334
-27.833333333333336
14.5
-27.833333333333336
[array([[ 26,  64,  30, 122]], dtype=int32), array([[  3, 122,  10,  46]], dtype=int32)]
14.5
14.5
-10.857142857142858
-25.357142857142858
14.5
-25.357142857142858
[array([[ 14, 112,  20,  44]], dtype=int32), array([[ 37,  71,  40, 131]], dtype=int32)]
-11.333333333333334
-11.333333333333334
20.0


[array([[ 22, 101,  61,  94]], dtype=int32), array([[  2,  89,  58, 101]], dtype=int32)]
-0.1794871794871795
-0.1794871794871795
0.21428571428571427
0.39377289377289376
-0.1794871794871795
0.39377289377289376
[array([[  4,  90,  61, 102]], dtype=int32)]
0.21052631578947367
0.21052631578947367
0.21052631578947367
[array([[ 16, 102,  60,  93]], dtype=int32), array([[  5,  90,  60, 102]], dtype=int32)]
-0.20454545454545456
-0.20454545454545456
0.21818181818181817
0.42272727272727273
-0.20454545454545456
0.42272727272727273
[]
[array([[  1,  87,  60, 101]], dtype=int32)]
0.23728813559322035
0.23728813559322035
0.23728813559322035
[array([[  9, 104,  61,  90]], dtype=int32), array([[  0,  85,  60, 100]], dtype=int32)]
-0.2692307692307692
-0.2692307692307692
0.25
0.5192307692307692
-0.2692307692307692
0.5192307692307692
[array([[ 15, 103,  63,  90]], dtype=int32), array([[  2,  84,  63, 102]], dtype=int32)]
-0.2708333333333333
-0.2708333333333333
0.29508196721311475
0.5659153005464481
-0.270

C:\Users\kvsp1\AppData\Local\Temp\ipykernel_14516\1146578054.py:135: RuntimeWarning: divide by zero encountered in scalar divide
  slope = (y2 - y1) / (x2 - x1)
C:\Users\kvsp1\AppData\Local\Temp\ipykernel_14516\1146578054.py:152: RuntimeWarning: divide by zero encountered in scalar divide
  slope = (y2 - y1) / (x2 - x1)


[array([[  1, 121,   1,  61]], dtype=int32), array([[ 17,  55,  31, 122]], dtype=int32), array([[18, 54, 58, 53]], dtype=int32), array([[  5, 120,   5,  58]], dtype=int32)]
-inf
-inf
4.785714285714286
inf
-0.025
-4.810714285714286
-inf
-inf
-inf
inf
-4.810714285714286
-inf
[array([[  9, 122,   9,  54]], dtype=int32), array([[  2, 115,   2,  59]], dtype=int32), array([[  5, 116,   6,  55]], dtype=int32), array([[19, 50, 64, 50]], dtype=int32), array([[ 27,  76,  36, 121]], dtype=int32)]
-inf
-inf
-inf
nan
-61.0
inf
0.0
61.0
5.0
5.0
-inf
nan
inf
61.0
5.0


...

[array([[  0, 106,  42,  93]], dtype=int32), array([[  0,  88,  55, 101]], dtype=int32)]
-0.30952380952380953
-0.30952380952380953
0.23636363636363636
0.5458874458874459
-0.30952380952380953
0.5458874458874459
[array([[  8, 103,  61,  88]], dtype=int32), array([[ 0, 86, 55, 99]], dtype=int32)]
-0.2830188679245283
-0.2830188679245283
0.23636363636363636
0.5193825042881647
-0.2830188679245283
0.5193825042881647
[array([[ 10, 102,  56,  89]], dtype=int32), array([[  0,  85,  59, 100]], dtype=int32)]
-0.2826086956521739
-0.2826086956521739
0.2542372881355932
0.536845983787767
-0.2826086956521739
0.536845983787767
[array([[  6, 106,  62,  89]], dtype=int32), array([[  1,  86,  59, 100]], dtype=int32)]
-0.30357142857142855
-0.30357142857142855
0.2413793103448276
0.5449507389162561
-0.30357142857142855
0.5449507389162561
[array([[  6, 107,  58,  91]], dtype=int32), array([[  5,  88,  60, 101]], dtype=int32)]
-0.3076923076923077
-0.3076923076923077
0.23636363636363636
0.544055944055944
-0.3076

[array([[ 13, 102,  61,  92]], dtype=int32), array([[  5,  88,  59, 100]], dtype=int32)]
-0.20833333333333334
-0.20833333333333334
0.2222222222222222
0.4305555555555556
-0.20833333333333334
0.4305555555555556
[array([[ 3, 88, 33, 95]], dtype=int32)]
0.23333333333333334
0.23333333333333334
0.23333333333333334
[array([[ 5, 87, 59, 99]], dtype=int32)]
0.2222222222222222
0.2222222222222222
0.2222222222222222
[array([[ 2, 87, 56, 99]], dtype=int32)]
0.2222222222222222
0.2222222222222222
0.2222222222222222
[array([[ 13, 102,  61,  92]], dtype=int32), array([[  5,  88,  59, 100]], dtype=int32)]
-0.20833333333333334
-0.20833333333333334
0.2222222222222222
0.4305555555555556
-0.20833333333333334
0.4305555555555556
[array([[  5,  88,  59, 100]], dtype=int32)]
0.2222222222222222
0.2222222222222222
0.2222222222222222
[array([[  9,  89,  60, 100]], dtype=int32)]
0.21568627450980393
0.21568627450980393
0.21568627450980393
[]
[array([[ 20, 100,  61,  92]], dtype=int32), array([[  4,  87,  61, 100]], 

[array([[  0,  54,   3, 123]], dtype=int32), array([[ 22,  73,  31, 120]], dtype=int32), array([[  4,  52,   6, 120]], dtype=int32)]
23.0
23.0
5.222222222222222
-17.77777777777778
34.0
28.77777777777778
23.0
-17.77777777777778
28.77777777777778
[array([[  0,  59,   2, 124]], dtype=int32), array([[  3,  57,   6, 119]], dtype=int32), array([[ 22,  80,  30, 124]], dtype=int32)]
32.5
32.5
20.666666666666668
-11.833333333333332
5.5
-15.166666666666668
32.5
-11.833333333333332
-15.166666666666668
[array([[  9,  61,  11, 123]], dtype=int32), array([[ 19,  84,  27, 124]], dtype=int32), array([[  2, 108,   2,  63]], dtype=int32), array([[20, 58, 59, 58]], dtype=int32)]
31.0
31.0
5.0
-26.0
-inf
-inf
0.0
inf
31.0
-26.0
-inf
inf
[array([[  3,  70,   4, 112]], dtype=int32), array([[18, 64, 60, 63]], dtype=int32)]
42.0
42.0
-0.023809523809523808
-42.023809523809526
42.0
-42.023809523809526
[array([[31, 70, 61, 70]], dtype=int32), array([[  0, 125,  45, 125]], dtype=int32)]
0.0
0.0
0.0
0.0
0.0
0.0
[a

[array([[ 11, 106,  60,  93]], dtype=int32), array([[  7,  91,  56, 101]], dtype=int32)]
-0.2653061224489796
-0.2653061224489796
0.20408163265306123
0.46938775510204084
-0.2653061224489796
0.46938775510204084
[array([[ 14, 105,  59,  94]], dtype=int32)]
-0.24444444444444444
-0.24444444444444444
-0.24444444444444444
[array([[ 17, 104,  61,  93]], dtype=int32), array([[  6,  89,  61, 101]], dtype=int32)]
-0.25
-0.25
0.21818181818181817
0.46818181818181814
-0.25
0.46818181818181814
[array([[ 19, 103,  57,  94]], dtype=int32)]
-0.23684210526315788
-0.23684210526315788
-0.23684210526315788
[array([[10, 89, 57, 98]], dtype=int32)]
0.19148936170212766
0.19148936170212766
0.19148936170212766
[array([[ 18, 103,  61,  93]], dtype=int32), array([[10, 88, 58, 97]], dtype=int32)]
-0.23255813953488372
-0.23255813953488372
0.1875
0.4200581395348837
-0.23255813953488372
0.4200581395348837
[array([[ 15, 105,  60,  94]], dtype=int32), array([[ 7, 88, 59, 98]], dtype=int32)]
-0.24444444444444444
-0.24444

[array([[ 10, 106,  60,  94]], dtype=int32), array([[ 6, 88, 56, 97]], dtype=int32)]
-0.24
-0.24
0.18
0.42
-0.24
0.42
[array([[ 12, 105,  57,  94]], dtype=int32), array([[10, 89, 59, 98]], dtype=int32)]
-0.24444444444444444
-0.24444444444444444
0.1836734693877551
0.42811791383219955
-0.24444444444444444
0.42811791383219955
[array([[  6, 107,  59,  94]], dtype=int32), array([[ 9, 89, 59, 98]], dtype=int32)]
-0.24528301886792453
-0.24528301886792453
0.18
0.4252830188679245
-0.24528301886792453
0.4252830188679245
[array([[ 12, 105,  60,  93]], dtype=int32), array([[ 5, 88, 55, 97]], dtype=int32)]
-0.25
-0.25
0.18
0.43
-0.25
0.43
[array([[  9, 106,  60,  93]], dtype=int32), array([[ 5, 88, 55, 97]], dtype=int32)]
-0.2549019607843137
-0.2549019607843137
0.18
0.4349019607843137
-0.2549019607843137
0.4349019607843137
[array([[ 10, 105,  60,  92]], dtype=int32), array([[ 8, 88, 55, 96]], dtype=int32)]
-0.26
-0.26
0.1702127659574468
0.43021276595744684
-0.26
0.43021276595744684
[array([[ 13, 10